In [21]:
import requests
from datetime import datetime
from time import strftime
from json import dumps

def get_forecast(
    latitude: float,
    longitude: float,
    time: str,
):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,precipitation_probability,precipitation,cloud_cover&timeformat=unixtime"
    response = requests.get(url)
    data = response.json()
    
    target_epoch = datetime.fromisoformat(time).timestamp()
    closest_index = min(range(len(data["hourly"]["time"])), key=lambda i: abs(data["hourly"]["time"][i] - target_epoch))
    
    return {
        "temperature": data["hourly"]["temperature_2m"][closest_index],
        "precipitation_probability": data["hourly"]["precipitation_probability"][closest_index]/100,
        "precipitation": data["hourly"]["precipitation"][closest_index],
        "cloud_cover": data["hourly"]["cloud_cover"][closest_index]/100,
    }

# Example for getting the current forecast for London
forecast = get_forecast(51.51, 0.13, strftime("%Y-%m-%d %H:%M"))
print(dumps(forecast, indent=4))

{
    "temperature": 15.5,
    "precipitation_probability": 0.02,
    "precipitation": 0.0,
    "cloud_cover": 1.0
}


In [22]:
from PBQA import DB, LLM
from time import strftime

class Agent:
    def __init__(self, db: DB, llm: LLM):
        self.db = db
        self.llm = llm
    
    def ask_weather(self, input: str):
        weather_query = self.llm.ask(
            input=input,
            pattern="weather",
            model="llama",
            external={"now": strftime("%Y-%m-%d %H:%M")}
        )
        
        print(f"Query:\n{dumps(weather_query, indent=4)}\n")
        
        forecast = get_forecast(**weather_query)
        forecast["temperature"] = f'{forecast["temperature"]} C'
        forecast["precipitation"] = f'{forecast["precipitation"]} mm'
        
        print(f"Forecast:\n{dumps(forecast, indent=4)}\n")
        
        return self.llm.ask(
            input=input,
            pattern="answer_json",
            model="llama",
            external={"json": dumps(forecast)}
        )

db = DB(path="db")
db.load_pattern("weather.yaml")
db.load_pattern("answer_json.yaml")

llm = LLM(db=db, host="192.168.0.137")
llm.connect_model(
    model="llama",
    port=8080,
    stop=["<|eot_id|>", "<|start_header_id|>"],
    temperature=0,
)

agent = Agent(db=db, llm=llm)
agent.ask_weather("Could I see the stars tonight?")

/opt/homebrew/Caskroom/miniconda/base/envs/test_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Metadata: {'system_prompt': "Your job is to translate the user's input into a weather query. The user may ask about the weather in general, the weather at a specific time, or the weather at a specific location. You should extract the time and location from the user's json and provide a (or part of a) weather query. Reply with the json for the weather query and nothing else.", 'now': {'external': True}, 'latitude': {'grammar': 'root         ::= coordinate\ncoordinate   ::= integer "." integer\ninteger      ::= digit | digit digit\ndigit        ::= "0" | "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9"\n'}, 'longitude': {'grammar': 'root         ::= coordinate\ncoord

Query:
{
    "latitude": 51.51,
    "longitude": 0.13,
    "time": "2024-06-19 22:00"
}

Forecast:
{
    "temperature": "14.5 C",
    "precipitation_probability": 0.01,
    "precipitation": "0.0 mm",
    "cloud_cover": 0.01
}



{'thought': 'The weather conditions are quite favorable for stargazing tonight, with a low temperature, low precipitation probability, and minimal cloud cover. It should be a great night to get out and enjoy the stars.',
 'answer': "Yes, the weather is perfect for stargazing tonight! The temperature is a comfortable 14.5°C, and there's only a 1% chance of precipitation. The cloud cover is also very low, making it an ideal night to get out and enjoy the stars."}